# WWYZ production lengths

## Setup

In [1]:
import numpy as np
import pandas as pd
import duckdb
from functions import zeroout, get_fatdiag

### Select MRIO version

In [2]:
# input, output = 'adb-mrio.parquet', 'lengths.parquet'
# input, output = 'adb-mrio62.parquet', 'lengths62.parquet'
input, output = 'adb-mrio62-const.parquet', 'lengths62-const.parquet'

### Parameters

In [3]:
sectors = pd.read_excel('../dicts/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Decompositions

In [4]:
DF = pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    y = np.sum(Y, axis=1)
    yd = get_fatdiag(Y)
    yf = y - yd
    v = np.where(x != 0, va / x, 0)
    Dx = np.diag(np.where(x != 0, 1 / x, 0))
    A = Z @ Dx
    Ad, Af = zeroout(A, inverse=True), zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)

    X = np.diag(v) @ B @ B @ np.diag(y)
    X_D = np.diag(v) @ Bd @ Bd @ np.diag(yd)
    X_RT = np.diag(v) @ Bd @ Bd @ np.diag(yf)
    Xd_GVC = np.diag(v) @ Bd @ Bd @ Af @ B @ np.diag(y)
    E_GVC = np.diag(v) @ B @ Af @ B @ np.diag(y)
    Xf_GVC = np.diag(v) @ Bd @ Af @ B @ Ad @ B @ np.diag(y)
    VY_D = np.diag(v) @ Bd @ np.diag(yd)
    VY_RT = np.diag(v) @ Bd @ np.diag(yf)
    VY_GVC = np.diag(v) @ Bd @ Af @ B @ np.diag(y)

    DFt = pd.DataFrame({
        't': int(year), 's': np.arange(1, G+1).repeat(N),
        'i': np.tile(sectors['ind'], G), 
        'i5': np.tile(sectors['ind5'], G), 
        'i15': np.tile(sectors['ind15'], G),
        'va': va, 'y': y,
        'Xv': np.sum(X, axis=1),
        'Xv_D': np.sum(X_D, axis=1),
        'Xv_RT': np.sum(X_RT, axis=1),
        'Xvd_GVC': np.sum(Xd_GVC, axis=1),
        'Ev_GVC': np.sum(E_GVC, axis=1),
        'Xvf_GVC': np.sum(Xf_GVC, axis=1),
        'V_D': np.sum(VY_D, axis=1),
        'V_RT': np.sum(VY_RT, axis=1),
        'V_GVC': np.sum(VY_GVC, axis=1),
        'Xy': np.sum(X, axis=0),
        'Xy_D': np.sum(X_D, axis=0),
        'Xy_RT': np.sum(X_RT, axis=0),
        'Xyd_GVC': np.sum(Xd_GVC, axis=0),
        'Ey_GVC': np.sum(E_GVC, axis=0),
        'Xyf_GVC': np.sum(Xf_GVC, axis=0),
        'Y_D': np.sum(VY_D, axis=0),
        'Y_RT': np.sum(VY_RT, axis=0),
        'Y_GVC': np.sum(VY_GVC, axis=0)
    })

    DFagg = DFt.drop(['i', 'i5', 'i15'], axis=1)
    DFagg.insert(2, 'agg', 0)
    DFagg.insert(3, 'i', 0)
    DFagg = DFagg.groupby(['t', 's', 'agg', 'i']).sum().reset_index()

    DF5 = DFt.drop(['i', 'i15'], axis=1)
    DF5.insert(2, 'agg', 5)
    DF5 = DF5.groupby(['t', 's', 'agg', 'i5']).sum().reset_index()
    DF5 = DF5.rename(columns={'i5': 'i'})

    DF15 = DFt.drop(['i', 'i5'], axis=1)
    DF15.insert(2, 'agg', 15)
    DF15 = DF15.groupby(['t', 's', 'agg', 'i15']).sum().reset_index()
    DF15 = DF15.rename(columns={'i15': 'i'})

    DF35 = DFt.drop(['i5', 'i15'], axis=1)
    DF35.insert(2, 'agg', 35)

    DF = pd.concat([DF, DFagg, DF5, DF15, DF35], ignore_index=True)
    
    print(f'{year} done')

2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


In [5]:
APL = pd.DataFrame({
    't': DF['t'], 's': DF['s'], 'agg': DF['agg'], 'i': DF['i'],
    'PLv': DF['Xv'] / DF['va'],
    'PLv_D': DF['Xv_D'] / DF['V_D'],
    'PLv_RT': DF['Xv_RT'] / DF['V_RT'],
    'PLvd_GVC': DF['Xvd_GVC'] / DF['V_GVC'],
    'CBv_GVC': DF['Ev_GVC'] / DF['V_GVC'],
    'PLvf_GVC': DF['Xvf_GVC'] / DF['V_GVC'],
    'PLy': DF['Xy'] / DF['y'],
    'PLy_D': DF['Xy_D'] / DF['Y_D'],
    'PLy_RT': DF['Xy_RT'] / DF['Y_RT'],
    'PLyd_GVC': DF['Xyd_GVC'] / DF['Y_GVC'],
    'CBy_GVC': DF['Ey_GVC'] / DF['Y_GVC'],
    'PLyf_GVC': DF['Xyf_GVC'] / DF['Y_GVC']
})

APL['PLv_GVC'] = APL['PLvd_GVC'] + APL['CBv_GVC'] + APL['PLvf_GVC']
APL['PLy_GVC'] = APL['PLyd_GVC'] + APL['CBy_GVC'] + APL['PLyf_GVC']
APL['GVC_POS'] = APL['PLv_GVC'] / APL['PLy_GVC']

APL.to_parquet(f'../data/{output}', index=False)

### View results

In [6]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{output}')").df()

,t,s,agg,i,PLv,PLv_D,PLv_RT,PLvd_GVC,CBv_GVC,PLvf_GVC,PLy,PLy_D,PLy_RT,PLyd_GVC,CBy_GVC,PLyf_GVC,PLv_GVC,PLy_GVC,GVC_POS
0,2007,1,0,0,2.181269,1.794026,2.082435,1.830896,1.335202,1.359788,2.046139,1.794026,2.082435,1.919056,1.368990,1.246887,4.525886,4.534933,0.998005
1,2007,2,0,0,2.011539,1.496777,1.641624,1.657048,1.356670,0.877161,1.976105,1.496777,1.641624,1.809195,1.384233,0.749471,3.890879,3.942899,0.986807
2,2007,3,0,0,2.148293,1.478859,1.747566,1.685176,1.382550,0.918672,2.076514,1.478859,1.747566,1.826164,1.359917,0.711213,3.986397,3.897294,1.022863
3,2007,4,0,0,2.076947,1.646354,1.783125,1.778284,1.359036,0.905389,2.290958,1.646354,1.783125,1.795243,1.308077,0.875290,4.042708,3.978610,1.016111
4,2007,5,0,0,1.897770,1.673917,2.107231,1.964856,1.296543,1.051616,1.849732,1.673917,2.107231,1.887912,1.325381,1.132858,4.313015,4.346150,0.992376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56443,2022,63,35,31,1.274692,1.162307,2.065152,2.402356,1.321332,1.083035,2.209337,1.787357,1.787357,1.814645,1.340744,1.472444,4.806723,4.627833,1.038655
56444,2022,63,35,32,1.184093,1.110956,1.485327,2.309829,1.273272,1.014111,2.164650,1.775112,1.775112,1.803972,1.315095,1.592498,4.597213,4.711564,0.975730
56445,2022,63,35,33,1.323933,1.219786,1.888040,2.565498,1.232490,0.835307,2.582774,2.068477,2.068477,1.823161,1.328810,1.391593,4.633294,4.543564,1.019749
56446,2022,63,35,34,2.287247,1.857467,2.605835,3.081860,1.318107,1.118911,2.607510,2.059663,2.059663,1.893458,1.313380,1.244608,5.518878,4.451446,1.239794
